In [1]:
import numpy as np
import pandas as pd
#laspy==1.7.0
#python=3.6

In [2]:
raw = pd.read_csv("my_point_cloud_new.csv", header=None).values

C:\Users\dany\.conda\envs\deepl_gpu\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw[0]

array(['Red', 'Green', 'Blue', 'X', 'Y', 'Z', 'Classification',
       'Intensity'], dtype=object)

In [4]:
raw[0:5]

array([['Red', 'Green', 'Blue', 'X', 'Y', 'Z', 'Classification',
        'Intensity'],
       ['81', '84', '94', '716000.5079999999', '935999.281',
        '10.609000000000002', '3', '0.0'],
       ['67', '65', '87', '716000.082', '935999.061', '11.248', '3',
        '0.0'],
       ['62', '61', '81', '716000.252', '935998.521', '4.348', '3',
        '0.0'],
       ['67', '65', '87', '716000.1390000001', '935998.958', '9.781',
        '3', '0.0']], dtype=object)

In [5]:
raw = raw[1:,:]

In [6]:
raw[0]

array(['81', '84', '94', '716000.5079999999', '935999.281',
       '10.609000000000002', '3', '0.0'], dtype=object)

In [7]:
labels = np.array(raw[:,6],dtype=np.int)
labels

array([3, 3, 3, ..., 7, 7, 7])

In [8]:
xyz = np.ascontiguousarray(raw[:, [3,4,5]], dtype=np.float32)

In [9]:
xyz

array([[7.1600050e+05, 9.3599925e+05, 1.0609000e+01],
       [7.1600006e+05, 9.3599906e+05, 1.1248000e+01],
       [7.1600025e+05, 9.3599850e+05, 4.3480000e+00],
       ...,
       [7.1685912e+05, 9.3598944e+05, 2.9100001e+00],
       [7.1669531e+05, 9.3595681e+05, 2.5699999e+00],
       [7.1681650e+05, 9.3594350e+05, 2.1700001e+00]], dtype=float32)

In [10]:
rgb = np.ascontiguousarray(raw[:, 0:3], dtype='uint8') * 256
rgb

array([[20736, 21504, 24064],
       [17152, 16640, 22272],
       [15872, 15616, 20736],
       ...,
       [32256, 29184, 28160],
       [ 7680,  6656, 15360],
       [33536, 27904, 30208]], dtype=uint16)

In [11]:
i = np.ascontiguousarray(raw[:, 7], dtype='float32') * 256
i

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [12]:
import laspy
hdr = laspy.header.Header(file_version=1.4, point_format=7)

In [13]:
mins = np.floor(np.min(xyz, axis=1))
hdr.offset = mins
hdr.scale = [0.01,0.01,0.01]
outfile = laspy.file.File("outfile7.las", mode="w", header=hdr)

In [14]:
outfile.X = xyz[:,0]
outfile.Y = xyz[:,1]
outfile.Z = xyz[:,2]
outfile.Red = rgb[:,0]
outfile.Green = rgb[:,1]
outfile.Blue = rgb[:,2]
outfile.Intensity = i
outfile.classification = labels
outfile.close()

In [15]:
inFile = laspy.file.File('outfile7.las', mode='r')

In [16]:
inFile.header.schema

In [17]:
inFile.Blue

array([24064, 22272, 20736, ..., 28160, 15360, 30208], dtype=uint16)

In [18]:
inFile.x

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
inFile.y

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
inFile.classification

array([3, 3, 3, ..., 7, 7, 7], dtype=uint8)

In [21]:
inFile.red

array([20736, 17152, 15872, ..., 32256,  7680, 33536], dtype=uint16)

In [22]:
inFile.Red

array([20736, 17152, 15872, ..., 32256,  7680, 33536], dtype=uint16)

In [23]:
inFile.X

array([716000, 716000, 716000, ..., 716859, 716695, 716816])

In [24]:
inFile.Y

array([935999, 935999, 935998, ..., 935989, 935956, 935943])

In [25]:
inFile.Z

array([10, 11,  4, ...,  2,  2,  2])

In [26]:
import numpy as np
points = np.vstack((inFile.X, inFile.Y, inFile.Z)).transpose()
colors = np.vstack((inFile.Red, inFile.Green, inFile.Blue)).transpose()

In [27]:
points

array([[716000, 935999,     10],
       [716000, 935999,     11],
       [716000, 935998,      4],
       ...,
       [716859, 935989,      2],
       [716695, 935956,      2],
       [716816, 935943,      2]])

In [28]:
colors

array([[20736, 21504, 24064],
       [17152, 16640, 22272],
       [15872, 15616, 20736],
       ...,
       [32256, 29184, 28160],
       [ 7680,  6656, 15360],
       [33536, 27904, 30208]], dtype=uint16)

In [29]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors/65535)
#pcd.normals = o3d.utility.Vector3dVector(normals)
o3d.visualization.draw_geometries([pcd])
print(np.array(pcd.colors))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[[0.31641108 0.32813001 0.3671931 ]
 [0.26172274 0.25391012 0.33984894]
 [0.2421912  0.23828489 0.31641108]
 ...
 [0.49219501 0.4453193  0.42969406]
 [0.11718929 0.10156405 0.23437858]
 [0.51172656 0.42578775 0.46094453]]


In [ ]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=0.40)
o3d.visualization.draw_geometries([voxel_grid])

In [56]:
import pptk
import numpy as np
v = pptk.viewer(points)
v.attributes(colors/65535)
v.color_map('cool')
v.set(point_size=0.001,bg_color=[0,0,0,0],show_axis=0,show_grid=0)